In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from os import listdir
from keras.models import load_model

import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from os.path import isfile, join

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, LSTM, TimeDistributed
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define the teacher model
teacher_model = load_model('/content/drive/MyDrive/Colab Notebooks/teacher_model.h5')
# Define the student model
student_model = keras.Sequential()
img_shape = (255,255,3)

student_model.add(Conv2D(32, kernel_size=(5,5), strides=1, padding='same', input_shape= img_shape, activation='relu'))
student_model.add(BatchNormalization())
student_model.add(MaxPooling2D(pool_size=(2,2)))

student_model.add(Conv2D(64, kernel_size=5 ,strides=1, padding='same', activation='relu'))
student_model.add(BatchNormalization())
student_model.add(MaxPooling2D(pool_size=(2,2)))

student_model.add(Dropout(0.3))
student_model.add(Flatten())

student_model.add(Dense(64, activation='relu'))
student_model.add(Dense(32, activation='relu'))
student_model.add(Dense(6, activation='softmax'))
# Define the temperature for distillation
temperature = 10
#Learning Rate
alpha = 0.1

In [ ]:
path='/content/drive/MyDrive/db2'
directory = listdir(path)
le = LabelEncoder()
le.fit(directory)
x = []
y = []
for dic in directory:
  imgpath = path+'/'+dic
  imgs = listdir(imgpath)
  for i in imgs:
    src = imgpath+'/'+i
    img = tf.keras.utils.load_img(src,target_size=(255,255,3))
    img=np.array(img)
    img=img/255.0
    x.append(img)
    y.append(dic)
y = le.transform(y)

In [ ]:
# Define the custom loss function for knowledge distillation
def knowledge_distillation_loss(y_true, y_pred):
    y_true_teacher, y_true_student = tf.split(y_true, num_or_size_splits=2, axis=0)
    y_pred_teacher, y_pred_student = tf.split(y_pred, num_or_size_splits=2, axis=0)

    soft_targets_pred = keras.activations.softmax(y_pred_teacher / temperature, axis=-1)
    soft_targets_true = keras.activations.softmax(y_true_teacher / temperature,axis=-1)
    #distillation loss using softmax 
    distillation_loss = keras.losses.kullback_leibler_divergence(soft_targets_pred, soft_targets_true)
    #student loss using hardlabels
    student_loss = keras.losses.categorical_crossentropy(y_true_student, y_pred_student)

    total_loss = (1 - alpha) * student_loss + alpha * temperature**2 * distillation_loss
    return total_loss

In [ ]:
# Define the training data and labels
x_train = x[:916]
y_train = y[:916]

# Split the training data into teacher and student inputs
x_train_teacher, x_train_student = tf.split(x_train, num_or_size_splits=2, axis=0)
y_train_teacher, y_train_student = tf.split(y_train, num_or_size_splits=2, axis=0)


In [ ]:
#Define the labels for knowledge distillation
y_train_teacher = teacher_model.predict(x_train_teacher)


15/15 [==============================] - 95s 6s/step


In [ ]:
print(y_train_teacher.shape)
print(y_train_student.shape)

(458, 6)
(458,)


In [ ]:
y_train_teacher

array([[1.4061455e-27, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.6679209e-01, 0.0000000e+00, 7.3320788e-01, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.2833966e-16, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [ ]:
y_train_teacher_new = np.zeros(458)

y_train_teacher_np = y_train_teacher

for i in range(0,len(y_train_teacher_np)):
  max_val=y_train_teacher_np[i][0]
  for j in range(1,6):
    y_train_teacher_new[i]=max(max_val,y_train_teacher_np[i][j])
  y_train_teacher_new[i]=max_val


In [ ]:
print((y_train_teacher_new.shape))
print((y_train_student))

y_train = tf.concat(tf.cast(tf.convert_to_tensor(y_train_teacher_new)), y_train_student,tf.float(),axis=1)

# student_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 
# student_model.fit(x_train,y_train, batch_size=128, epochs=10, validation_split=0.2)

(458,)
tf.Tensor(
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4], shape=(458,), dtype=int64)


TypeError: ignored